In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predict-energy-behavior-of-prosumers/client.csv
/kaggle/input/predict-energy-behavior-of-prosumers/gas_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/electricity_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/weather_station_to_county_mapping.csv
/kaggle/input/predict-energy-behavior-of-prosumers/public_timeseries_testing_util.py
/kaggle/input/predict-energy-behavior-of-prosumers/historical_weather.csv
/kaggle/input/predict-energy-behavior-of-prosumers/county_id_to_name_map.json
/kaggle/input/predict-energy-behavior-of-prosumers/train.csv
/kaggle/input/predict-energy-behavior-of-prosumers/forecast_weather.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/sample_submission.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/client.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/gas_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/electricity

In [2]:
import enefit
env = enefit.make_env()
iter_test = env.iter_test()

In [3]:
import os
import gc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.model_selection import cross_val_score

import xgboost as xgb
import lightgbm as lgb
import catboost as cb

import optuna
import shap

In [4]:
def feature_eng(df):
    # used for transforming new / test /prediction data only because of difference in column name prediction_datetime vs datetime

    df["prediction_datetime"] = pd.to_datetime(df["prediction_datetime"])

    df["month"]  = df["prediction_datetime"].dt.month
    df["day"]     = df["prediction_datetime"].dt.day
    df["weekday"] = df["prediction_datetime"].dt.weekday
    df["hour"]    = df["prediction_datetime"].dt.hour

    df["county"]       = df["county"].astype("category")
    df["is_business"]  = df["is_business"].astype("category")
    df["product_type"] = df["product_type"].astype("category")
    df["is_consumption"] = df["is_consumption"].astype("category")
    
    
    df = df.set_index(["row_id"])
    df = df.drop(columns=["prediction_datetime", "prediction_unit_id", "currently_scored"])

    return df

In [5]:
df_train = pd.read_csv(r'/kaggle/input/predict-energy-behavior-of-prosumers/train.csv')

df_train["datetime"] = pd.to_datetime(df_train["datetime"])

df_train["month"] = df_train["datetime"].dt.month
df_train["day"] = df_train["datetime"].dt.day
df_train["weekday"] = df_train["datetime"].dt.weekday
df_train["hour"] = df_train["datetime"].dt.hour

df_train["county"] = df_train["county"].astype("category")
df_train["is_business"] = df_train["is_business"].astype("category")
df_train["product_type"] = df_train["product_type"].astype("category")
df_train["is_consumption"] = df_train["is_consumption"].astype("category")


df_train = df_train.set_index(["row_id", "data_block_id"])
df_train = df_train.drop(columns=["datetime", "prediction_unit_id"])
columns_to_keep = ['county', 'is_business', 'product_type', 'is_consumption', 'month', 'day', 'weekday', 'hour', 'target']
df_train = df_train[columns_to_keep]
df_train

,,county,is_business,product_type,is_consumption,month,day,weekday,hour,target
row_id,data_block_id,,,,,,,,,
0,0,0,0,1,0,9,1,2,0,0.713
1,0,0,0,1,1,9,1,2,0,96.590
2,0,0,0,2,0,9,1,2,0,0.000
3,0,0,0,2,1,9,1,2,0,17.314
4,0,0,0,3,0,9,1,2,0,2.904
...,...,...,...,...,...,...,...,...,...,...
2018347,637,15,1,0,1,5,31,2,23,197.233
2018348,637,15,1,1,0,5,31,2,23,0.000
2018349,637,15,1,1,1,5,31,2,23,28.404


In [6]:
df_train = df_train.dropna(subset=["target"])

In [7]:
X_train, y_train = df_train.drop('target', axis=1), df_train['target']

In [8]:
from xgboost import XGBRegressor
model = XGBRegressor()
model.fit(X_train.to_numpy(), y_train.to_numpy())

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [9]:
counter = 0
for (test, revealed_targets, client, historical_weather,
        forecast_weather, electricity_prices, gas_prices, sample_prediction) in iter_test:

    X_test = feature_eng(test)
    y_pred = model.predict(X_test)
    y_pred = y_pred.clip(0)
    
    sample_prediction['target'] = y_pred
    counter += 1
    env.predict(sample_prediction)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
